In [1]:
########
## Env Setup / Preamble
########
%reload_ext autoreload
%autoreload complete 

%reload_ext Cython

%display latex

################

import cython
from sage.all import *

################

import tempfile
import mmap
from multiprocessing import shared_memory

################

import numpy as np
import scipy.constants

np.random.seed( int(scipy.constants.golden_ratio * 1.e6) )

from numpy import vectorize
from numpy import r_, c_, s_

################

####    boiler-plate class for named constants
from sage.symbolic.constants import Constant
class ConstantFromExpression(Constant):
    def __init__(self, name, v):
        conversions = dict(maxima=repr(maxima.coerce(v)))
        Constant.__init__(self, name, conversions=conversions)
        self._v = v
    def __float__(self): return float(self._v)
    def __complex__(self): return complex(self._v)
    def _mpfr_(self, R): return R(self._v)
    def _real_double_(self, R): return R(self._v)

def NamedExpression(name, expr):
    return ConstantFromExpression(name,expr).expression()
####

import sage.symbolic.units as units     # <: TODO

𝛑   = NamedExpression('𝛑', scipy.constants.pi)         # pi

𝙘 = NamedExpression('𝙘', scipy.constants.speed_of_light)

𝛆_0 = NamedExpression('𝛆_0', scipy.constants.epsilon_0) # electric permittivity
𝛍_0 = NamedExpression('𝛍_0', scipy.constants.mu_0)     # magnetic permeability of vacuum
𝛋 = 1. / (4 * 𝛑 * 𝛆_0)      # Coulomb's constant in [ N·m²/C² ]
𝙦_e = NamedExpression('𝙦_e', -scipy.constants.elementary_charge)

𝒉_planck = NamedExpression('𝒉', scipy.constants.Planck)
ℏ = NamedExpression('ℏ', scipy.constants.hbar)

𝛂 = NamedExpression('𝛂', scipy.constants.fine_structure)

𝒎_e = NamedExpression('𝒎_e', scipy.constants.electron_mass)
𝒎_p = NamedExpression('𝒎_p', scipy.constants.proton_mass)

amu = NamedExpression('amu', scipy.constants.atomic_mass)

################

#Parallelism().set(nproc=8)
print(f"Python: {sys.version}")
version()


Python: 3.12.5 (main, Feb 25 2025, 12:37:41) [Clang 16.0.0 (clang-1600.0.26.6)]


'SageMath version 10.6, Release Date: 2025-03-31'

## defn.s

Minkowski Tensor, positive signature

$$ 
\eta_{\mu\nu} = 
\begin{bmatrix} 
    c^2  & 0 & \cdots & 0 \\
     0 & -1 & \ddots & \vdots \\
    \vdots  & \ddots  & -1 & 0 \\
       0    & \cdots   & 0 & -1 \\
\end{bmatrix} 
$$

Riemann Tensor:

$$R^\alpha_{\beta\mu\nu} = -\Gamma^\alpha_{\beta\mu,\nu} + \Gamma^\alpha_{\beta\nu,\mu} + \Gamma^\alpha_{\sigma\mu} \Gamma^\sigma_{\beta\nu} - \Gamma^\alpha_{\sigma\nu} \Gamma^\sigma_{\beta \mu}$$

Ricci Curvature Tensor:

$$R_{\mu \nu} = R^\alpha_{\mu\nu\alpha}$$

Ricci Curvature Scalar

$$ R = R^\mu_\mu $$


Einstein Tensor

$$G_{\mu\nu} = R_{\mu\nu} - \frac{1}{2} R g_{\mu\nu}$$

$$\kappa = \frac{8\pi G}{c^4}$$

Einstein-Hilbert Field Equations

$$ 
\begin{aligned}
R_{\mu\nu} - \frac{1}{2} g_{\mu\nu} R + \Lambda g_{\mu\nu} &= 
    \frac{8 \pi G}{c^4} T_{\mu\nu} 
\\ \quad \Leftrightarrow \quad 
G_{\mu\nu} + \Lambda g_{\mu\nu} &= \kappa T_{\mu\nu}
\\ \quad \Leftrightarrow \quad 
R_{\mu\nu} - \Lambda g_{\mu\nu} &= \kappa \left( T_{\mu\nu} - \frac{1}{2} T g_{\mu\nu}\right)
\end{aligned}
$$

---

# [1]

# [2]

# [3]

# [4]

# [5]

# [6]

# [7]

do a rotation on a 2D metric (like the circle/polar problem) and show same answer

$(ds)^2 =  (dr)^2 + R^2 (d \theta)^2$

rotation matrix 
$$
B = \begin{bmatrix}
\cos(\theta) & \sin(\theta) \\
-\sin(\theta) & \cos(\theta)
\end{bmatrix}
$$

$$
A = \begin{bmatrix}
1 & 0 \\
0 & R^2
\end{bmatrix}
$$

suppose I have an $A^\prime = B * A * B^T$, then we seek to show that a scalar $s$ 
resulting from $s = (c_1\ c_2) A \begin{pmatrix} c_1 \\ c_2 \end{pmatrix}$ does not change
after the rotation (i.e., the normed length $\|s\| = \|\|$)

So then 
$$ 
A^\prime = 
\begin{bmatrix}
\cos(\theta) & \sin(\theta) \\
-\sin(\theta) & \cos(\theta)
\end{bmatrix} 
\cdot
\begin{bmatrix}
1 & 0 \\
0 & R^2
\end{bmatrix}
\cdot
\begin{bmatrix}
\cos(\theta) & -\sin(\theta) \\
\sin(\theta) & \cos(\theta)
\end{bmatrix}
$$

$$
s^\prime = 
(c_1\ c_2)
\begin{bmatrix}
\cos(\theta) & -\sin(\theta) \\
\sin(\theta) & \cos(\theta)
\end{bmatrix}

A^\prime 

\begin{bmatrix}
\cos(\theta) & \sin(\theta) \\
-\sin(\theta) & \cos(\theta)
\end{bmatrix} 
\cdot
\begin{pmatrix} c_1 \\ c_2 \end{pmatrix}
$$

so then; 

$$
s^\prime = 
(c_1\ c_2)
\begin{bmatrix}
\cos(\theta) & -\sin(\theta) \\
\sin(\theta) & \cos(\theta)
\end{bmatrix}

\begin{bmatrix}
\cos(\theta) & \sin(\theta) \\
-\sin(\theta) & \cos(\theta)
\end{bmatrix} 
\cdot
\begin{bmatrix}
1 & 0 \\
0 & R^2
\end{bmatrix}
\cdot
\begin{bmatrix}
\cos(\theta) & -\sin(\theta) \\
\sin(\theta) & \cos(\theta)
\end{bmatrix}

\begin{bmatrix}
\cos(\theta) & \sin(\theta) \\
-\sin(\theta) & \cos(\theta)
\end{bmatrix} 
\cdot
\begin{pmatrix} c_1 \\ c_2 \end{pmatrix}
$$

then; 

$$ s^\prime = 
(c_1\ c_2)

A

\begin{pmatrix} c_1 \\ c_2 \end{pmatrix}
$$

or; 

$$ s^\prime = s $$

# [8]

# [9]

# [10]

# [11]

# [12]

# [13]

# [14]

Figure out the metric for a 2D curved earth with longitude/latitude coordinates.

In [2]:
var("R")
S2 = manifolds.Sphere(2, radius=R)
R3 = S2.ambient()

spherical.<𝛉, 𝛗> = S2.spherical_coordinates()
spherical


Chart (A, (theta, phi))

In [3]:
# South Pole, 𝛉 = 𝛑
spherical.coord_range()


theta: (0, pi); phi: [-pi, pi] (periodic)

In [4]:
cartesian.<x,y,z> = R3.chart()

cartesian


Chart (E^3, (x, y, z))

In [5]:
𝚽 = S2.embedding()
𝚽.display()


iota: S^2_R → E^3
on A: (theta, phi) ↦ (x, y, z) = (R*cos(phi)*sin(theta), R*sin(phi)*sin(theta), R*cos(theta))

In [6]:
g = S2.induced_metric()
g[:]


[             R^2                0]
[               0 R^2*sin(theta)^2]

In [7]:
g.display()


g = R^2 dtheta⊗dtheta + R^2*sin(theta)^2 dphi⊗dphi

In [8]:
# TODO :> need to show metric `g` in terms of <𝛌,l>


# [15]

Find EoM for and solve it for Denver to London

---

Recall that the interval path length and metric tensor relation is given by:

$$ ds^2 = g_{\mu\nu}\ dx^\mu dx^\nu $$

which for our purposes in this problem, we will phrase as:

$$ ds^2 = R^2\ d\theta^2 + R^2\ \sin^2\theta\ d\phi^2 $$

For now, we can assume $R = 1$, which simplifies to 

$$ ds^2 = (1)\ d\theta^2 + (\sin^2\theta)\ d\phi^2 $$


Now recall that the Geodesic Equation is given by 

$$ 
    \frac{\partial^2}{\partial \tau^2} x^\mu +  
        \Gamma^{\mu}_{\alpha \beta} 
        \frac{\partial x^\alpha}{\partial \tau}
        \frac{\partial x^\beta}{\partial \tau}
        = 0
    \\ \Leftrightarrow \\
    \ddot{\gamma}^\mu + \Gamma^{\mu}_{\alpha \beta}
    \dot{\gamma}^\alpha \dot{\gamma}^\beta
    = 0 
$$

where the "proper-time" $\tau = s$, our implicit path interval for the given metric. 

Recall the definition of the Christoffel symbols of the second kind (the affine connection coefficients), 

$$ \Gamma^\alpha_{\mu\nu} = \frac{1}{2} g^{\alpha \beta} \left( g_{\beta\mu,\nu} + g_{\nu\beta,\mu} - g_{\mu\nu,\beta} \right) $$

For our given system, the Christoffel symbols $\Gamma^\mu_{\alpha\beta}$ are:

In [9]:
g.christoffel_symbols_display(
    coordinate_labels=False,
    only_nonredundant=False
)


Gam^1_22 = -cos(theta)*sin(theta) 
Gam^2_12 = cos(theta)/sin(theta) 
Gam^2_21 = cos(theta)/sin(theta)

Or equivalently, 

$$\begin{aligned} 
\Gamma^\theta_{\phi\phi} &= -\sin\left(\theta\right)\cos\left(\theta\right) \\
\Gamma^\phi_{\theta\phi} = \Gamma^\phi_{\phi\theta} = \cot\left(\theta\right)
    &\quad \Leftrightarrow \quad
\Gamma^\phi_{\alpha\beta} = 2 \cot\theta
\end{aligned}$$

We may now simplify the geodesic equation, indexed by $\mu \in \{\theta, \phi\}$, to receive the 
equations of motion in the coordinate system: 

$$
\begin{array}{rlcc}
    \ddot{\theta}\ &-\ \sin\left(\theta\right)\cos\left(\theta\right) &\left(\dot\phi\right)^2 &= 0 \\
    \ddot{\phi}\ &+\ 2 \cot\left(\theta\right) &\dot\theta\dot\phi &= 0
\end{array} 
$$

It will now be useful to us to specify the transformed coordinates for both cities,
on the 2-Sphere: 

In [10]:
# South Pole, West - Negative Sign 
# North Pole, East - Positive Sign
l_𝛉 = lambda l: (𝛑/2) - ((𝛑 / 180.) * l)    # latitude `l` to 𝛉
𝛌_𝛗 = lambda 𝛌: ((𝛑 / 180.) * 𝛌)            # longitude `𝛌` to 𝛗


In [11]:
denver = S2.point( 
    (
        l_𝛉(39.73915),  # 39.73915°N
        𝛌_𝛗(-104.9847), # 104.9847°W
    ), 
    name="Denver"
)

print(denver)
print(f"𝛉: {n(spherical(denver)[0], digits=5)}\t 𝛗: {n(spherical(denver)[1], digits=5)}")
spherical(denver)


Point Denver on the 2-sphere S^2_R of radius R smoothly embedded in the Euclidean space E^3
𝛉: 0.87722	 𝛗: -1.8323


(0.279226944444444*𝛑, -0.583248333333333*𝛑)

In [12]:
london = S2.point( 
    (
        l_𝛉(51.50853), # 39.73915°N
        𝛌_𝛗(-0.12574)  # 104.9847°W 
    ), 
    name="London"
)

print(london)
print(f"𝛉: {n(spherical(london)[0], digits=5)}\t 𝛗: {n(spherical(london)[1], digits=5)}")
spherical(london)


Point London on the 2-sphere S^2_R of radius R smoothly embedded in the Euclidean space E^3
𝛉: 0.67180	 𝛗: -0.0021946


(0.213841500000000*𝛑, -0.000698555555555556*𝛑)

---

Let's stop and take a second to make sure that we understand the problem we trying to solve.
The differential system is of order-2, nonlinear, coupled, homogeneous, ordinary differential 
equations.  

We will introduce an additional function $L(s) = s$ to additionally solve for 
the path-length interval, with $L(s=L_0) = L_0$ for constant $L_0$. 

The problem presented is a Two-Point Boundary Value Problem, which we will solve 
with the "Shooting Method", numerically, the technique common in Partial 
Differential Equations. 

$$
\left[
    \begin{array}{rcl}
        \ddot{\theta}\ &-\ \sin\left(\theta\right)\cos\left(\theta\right) &\left(\dot\phi\right)^2 \\
        \ddot{\phi}\ &+\ 2 \cot\left(\theta\right) &\dot\theta\dot\phi
    \end{array}
\right] = \begin{pmatrix} 0 \\ 0 \end{pmatrix}
$$

subject to boundary conditions

$$
\left\{
\begin{array}{rcrcrcr}
    \theta(s=0) &\approx&   0.87722 & & \theta(s=L_0)   &\approx&   0.67180 
    \\
    \phi(s=0)   &\approx&   -1.8323 & & \phi(s=L_0)     &\approx&   0.0021946
    \\
\end{array}
\right\}
$$

In [13]:
var('s')

_𝛉 = function('𝛉')(s)
_𝛗 = function('𝛗')(s)


In [14]:
_𝛉ʺ = diff(_𝛉, s, 2) - (sin(_𝛉)*cos(_𝛉)) == 0
_𝛉ʺ


-cos(𝛉(s))*sin(𝛉(s)) + diff(𝛉(s), s, s) == 0

In [15]:
_𝛗ʺ = diff(_𝛗, s, 2) + (2 * cot(_𝛉)) == 0
_𝛗ʺ


2*cot(𝛉(s)) + diff(𝛗(s), s, s) == 0

However, there are some additional stipulations and mathematical analysis that 
we need to make, to render this equation into something more workable. 

Let $\left\{ u = \dot{\theta},\ v = \dot{\phi} \right\}$, and 
$\left\{ w = \sin{\theta}, \dot{w} = \cos{\theta} \right\}$, then:

$$
\left[
    \begin{array}{rcl}
        \dot{u}\ &-\ v^2 w &\dot{w} \\
        \dot{v}\ &+\ 2 u v \frac{1}{w} &\dot{w}  
    \end{array}
\right] = \begin{pmatrix} 0 \\ 0 \end{pmatrix}
$$


Now, let's include the earlier expression for the metric, $(\Delta s)^2 = (\Delta \theta)^2 + (\Delta \phi)^2 \sin^2 \theta$
 as an additional equation in the system to solve.  

We know that the Geodesic Equation uniquely solves the minimum-path interval $s$,
for the Parallel Transport problem, and this situation applies in this case (since 
we are necessarily traveling along a geodesic, the above system _is equivalent_ 
to the geodesic system that we wish to solve). 

Then, 

$$
\left[
    \begin{array}{rcr}
        \ddot{\theta}\ &-&\ u\ \dot{u}\ (v^2)   \\
        \dot{v}\ &+&\ 2\ (1/u)\ v \dot\theta    \\
        
    \end{array}
\right] = \begin{pmatrix} 0 \\ 0 \end{pmatrix}
$$

---

Equivalently, we may engage the computer algebra system using the Manifolds 
construction as follows:

In [16]:
var('s')

v = S2.tangent_vector(denver,   # point
                      (         # direction, ```London - Denver```
                          spherical(london) + 
                          (-1 * spherical(denver))
                      ), 
)

[s, s_min, s_max] = var('s smin smax')

geodesic = S2.integrated_geodesic(
    g,
    (s, s_min, s_max),
    v,
    chart=spherical,
    name="geodesic"
)

geo = geodesic.system(verbose=True)
geo


Geodesic geodesic in the 2-sphere S^2_R of radius R smoothly embedded in the Euclidean space E^3 equipped with Riemannian metric g on the 2-sphere S^2_R of radius R smoothly embedded in the Euclidean space E^3, and integrated over the Real interval (smin, smax) as a solution to the following geodesic equations, written with respect to Chart (A, (theta, phi)):

Initial point: Point Denver on the 2-sphere S^2_R of radius R smoothly embedded in the Euclidean space E^3 with coordinates [0.279226944444444*𝛑, -0.583248333333333*𝛑] with respect to Chart (A, (theta, phi))
Initial tangent vector: Tangent vector at Point Denver on the 2-sphere S^2_R of radius R smoothly embedded in the Euclidean space E^3 with components [0.213841500000000*𝛑, -0.000698555555555556*𝛑] with respect to Chart (A, (theta, phi))

d(theta)/ds = Dtheta
d(phi)/ds = Dphi
d(Dtheta)/ds = Dphi^2*cos(theta)*sin(theta)
d(Dphi)/ds = -2*Dphi*Dtheta*cos(theta)/sin(theta)



[[Dphi^2*cos(theta)*sin(theta), -2*Dphi*Dtheta*cos(theta)/sin(theta)],
 Tangent vector at Point Denver on the 2-sphere S^2_R of radius R smoothly embedded in the Euclidean space E^3,
 Chart (A, (theta, phi))]

---

For funsies, we can also analytically solve the computer algebra form of the geodesic system of equations of motion, and express them as the Lagrangian of an Initial Value Problem in the coordinate system: 

In [17]:
sol = geodesic.solve_analytical()
expand(simplify(sol))


(ilt((g4545*theta_0 + Dtheta_0 + laplace(cos(y1(s))*sin(y1(s))*y4(s)^2, s, g4545))/g4545^2, g4545, s),
 ilt((g4545*phi_0 + Dphi_0 - 2*laplace(cos(y1(s))*y3(s)*y4(s)/sin(y1(s)), s, g4545))/g4545^2, g4545, s))

# [16]

Calculate the Reimann curvature tensor for a flat space in polar coordinates and on the sphere S2.

In [18]:
Γ = g.connection()

𝓡 = Γ.riemann()

𝓡.display_comp()


Riem(g)^theta_phi,theta,phi = sin(theta)^2 
Riem(g)^theta_phi,phi,theta = -sin(theta)^2 
Riem(g)^phi_theta,theta,phi = -1 
Riem(g)^phi_theta,phi,theta = 1

In [19]:
𝓡.display()


Riem(g) = sin(theta)^2 ∂/∂theta⊗dphi⊗dtheta⊗dphi - sin(theta)^2 ∂/∂theta⊗dphi⊗dphi⊗dtheta - ∂/∂phi⊗dtheta⊗dtheta⊗dphi + ∂/∂phi⊗dtheta⊗dphi⊗dtheta

Now, let's compare this to a "flat" space (such as the Minkowski metric )

# [17]

Compute the change in a vector due to transport around closed curve in S2

The "Change in vector produced by parallel transport around a closed path", from 
Table 6.1, can be stated as:

$$\Delta A^\alpha = R^\alpha_{\beta\mu\nu} A^\beta d\xi^\nu d\zeta^\mu$$

---

but instead, we're going to do it the following way: 

!!!!!!!

Use $\delta A^\mu = -\Gamma^\mu_{\nu\beta} A^\nu (d x)^\beta$ and the metric for the 2-sphere
(assuming radius $R=1$): 

$$g_{\mu\nu} = (d\theta)^2 + \sin^2 (\theta)\ (d\phi)^2$$

Recall the Christoffel symbols for the 2-Sphere $\mathbb{S}^2$ are:

In [21]:
g.christoffel_symbols_display(
    coordinate_labels=True,
    only_nonredundant=False
)


Gam^theta_phi,phi = -cos(theta)*sin(theta) 
Gam^phi_theta,phi = cos(theta)/sin(theta) 
Gam^phi_phi,theta = cos(theta)/sin(theta)

The system of equations is then:

$$\begin{aligned}
\delta A^\theta &= -(-\sin\theta\cos\theta) A^\phi (d\phi) \\
\delta A^\phi &= [-\cot\theta A^\phi (d\theta)] + [-\cot\theta A^\theta (d\phi)]
\end{aligned}$$

which simplifies to

$$\begin{aligned}
\delta A^\theta &= (\sin\theta\cos\theta) A^\phi (d\phi) \\
\delta A^\phi &= -2\cot\theta [A^\phi d\theta + A^\theta d\phi]
\end{aligned}$$

Incidentally, $A^\phi d\theta$ and $A^\theta d\phi$ must equal $\cdots$; and $A^\phi d\phi$ must be $\cdots$. 

[... more detail needed here ...]

Thus, the deflection $\delta A^\mu$ is equivalent to the induced metric!

We can conclude that on the metric surface of a 2-Sphere, deflection in the orientation of a vector, parallel-transported around a closed curve, arises directly from the induced metric of the surface. In this case of non-zero intrinsic curvature (Riemann tensor)

So, for example, say we are traveling in a open curve path going from the "North Pole" to the "South Pole".

Then, our $d\phi = 0$, and we will have to integrate over $d\theta$ using a forward-Euler update scheme, over the 
range $d\theta \in (0, \pi)$.

Our system reduces to: 

$$\begin{aligned}
\delta A^\theta &= 0 \\
\delta A^\phi &= A^\phi (-2\cot\theta\ d\theta)
\end{aligned}$$

But, if we integrate over $d \theta$, the boundary-condition limits of our integral have singular values
at the poles of $\mathbb{S}^2$, so we will instead consider the domain $(\epsilon, \pi - \epsilon)$ for
an arbitrary limit $\lim\limits{\epsilon \rightarrow 0}$ for $\epsilon > 0$. 

It follows that the deflection of the vector $A$ parallel-transported along this curve can be given by the system 

$$\begin{aligned}
\Delta A^\theta &= 0 \\
\Delta A^\phi &= -2 A^\phi \int_{\epsilon}^{\pi - \epsilon} \cot\theta\ d\theta
\end{aligned}$$

so then; 

$$ \Delta A^\phi = -2 A^\phi \left[ \ln|\sin\theta| \right]\rvert_{\epsilon}^{\pi - \epsilon} $$

which simplifies to 

$$
\Delta A^\phi = -2 A^\phi\ \left[ \ln|\sin(\pi-\epsilon)| - \ln|\sin(\epsilon)| \right] \\
\Rightarrow  \\
\Delta A^\phi = -2 A^\phi\ \left[ \ln\lvert \frac{\sin(\pi-\epsilon)}{\sin(\epsilon)} \rvert\right]
$$



We can then perform a small angle approximation on $\epsilon$ via taylor series expansion around the point $\theta = \pi$ 
to resolve the numerator of the "..." of the logarithmic expression. Since $\mathrm{sine}$ is a periodic function, the
fractional expression above reduces to exactly $\frac{\sin(\pi - \epsilon)}{\sin(\epsilon)} = 1$. 

[Note :> Skipped formal steps here]

The polynomial expressions of this Taylor series expansion at $\theta = 0, \pi$ will result in something similar to

$$\Delta A^\phi = -2 A^\phi\ \ln\lvert 1 \rvert$$

However, $\ln(1) = $, so our total deflection traveling from pole-to-pole is $\mathbf{0}$!

This would be different if we chose a non-trivial path, such as the closed curve around the sphere 
at the latitude $5^\degree \mathrm{N}$.

# [18]

Special Relativity: "car in a barn paradox" - Time Dilation and Length Contraction. 

>   The lack of a notion of absolute simultaneity in special relativity leads to many supposed paradoxes. One of the most famous of these involves a car and a garage of equal proper length. The driver speeds toward the garage, and a doorman at the garage is instructed to slam the door shut as soon as the back end of the car enters the garage. According to the doorman, "the car Lorentz contracted and easily fitted into the garage when I slammed the door." According to the driver, "the garage Lorentz contracted and was too small for the car when I entered the garage." Draw a spacetime diagram showing the above events and explain what really happens. Is the doorman's statement correct? Is the driver's statement correct? For definiteness, assome that the car crashes through the back wall of the garage without stopping or slowing down. 

We are supposed to start with expressions for the Lorentz Transforms (Lorentz Boosts + Rotations) 
to derive the following expressions, in the Lorentz coordinates (after the boost transform) on the Poincare half-disk. 

Let 

- $\beta = (\nu/c)$, 
- $\gamma = \sqrt{1 - \beta^2}$

then

$$
\begin{bmatrix}
    ct^\prime \\ x^\prime \\ y^\prime \\ z^\prime
\end{bmatrix}
=
\begin{bmatrix}
    \gamma          &   -\gamma\beta    &   0   &   0 \\
    -\gamma\beta    &   \gamma          &   0   &   0 \\
    0               &   0               &   1   &   0 \\
    0               &   0               &   0   &   1
\end{bmatrix}
\cdot
\begin{bmatrix}
    ct \\ x \\ y \\ z
\end{bmatrix}
$$

We want to arrive at the expressions for Special Relativity from Modern Physics. 

$$\begin{align*}
    L       &= \gamma L_0 \\ 
            &= L_0 \sqrt{1 - \beta^2} \\
            \\
    \tau    &= \gamma^{-1} t_0 \\
            &= \frac{t_0}{\sqrt{1 - \beta^2}}
\end{align*}$$

---

We can start by acknowledging that 

$$ 
L_0 = \sqrt{x^2 + y^2 + z^2} \\
L = \sqrt{(x^\prime)^2 + (y^\prime)^2 + (z^\prime)^2}
$$

Let's proceed by computing $L$

$$\begin{aligned}
    L   &=  \sqrt{(x^\prime)^2 + (y^\prime)^2 + (z^\prime)^2} \\
        &=  \sqrt{()}
\end{aligned}$$

---

Both observers are correct, both have their own frame of reference (with associated 
inertial coordinate systems/frames). 

Let's split the coordinate expressions for each frame of reference, expressed in
terms of the absolute coordinate time; i.e., $\tau_c$ is the car's time-coordinate 
in its own frame w.r.t. coordinate time $t$ (the Driver Observer), and $\tau_g$ 
is the garage's time coordinate w.r.t. coordinate time $t$ (The Doorman Observer). 

Then, we can add "markers" for the events in coordinate time, say $t_0$ for when 
the garage door closes, and $t_1$ for when the car hits the back of the garage. 
Since each Observer is in their own inertial frame, the other "object" appears to
be moving _relative_ to each Observer with some velocity. 

The Doorman Observer will attempt to close the garage door when the rear of the car crosses
a threshold associated with time $t_0$, but due to length contraction and time dilation,
this will occur "too early", i.e., the back of the car has not actually yet crossed
the needed threshold.  From the Driver's Observation perspective, the garage door
will close "too late", as in, they will have already hit the back of the garage
(event $t_1$) by the time the Doorman observes the Driver crossing the threshold
to engage the event $t_0$. 

In this sense, both accounts are correct, because each Observer encounters 
the events $t_0$ and $t_1$ (in absolute coordinate time) in a different order --
The Doorman closes the garage door (event $t_0$) before observing the car hitting 
the back of the garage ($t_1$), while the Driver observes hitting the back of the
garage (event $t_1$) before the garage door closes (event $t_0$). 


# [19]

You watch a muon speed to earth with energy 100 GeV. How long will it take to decay?  To reach the ground from the upper atmosphere?

Use the slide from the end of Class 23 lecture notes!

A muon's rest mass is 207 times the rest-mass of an electron, and decays 
(in its own time-frame) in 2.2 $\mu s$.  

We will assume that the distance travelled from the ionosphere to the surface is
48 km. 

We need to apply 

$$ E = \sqrt{(mc^2)^2 + p^2} $$

observer at ground level, given total energy and rest mass to calculate momentum
expression of muon, then use Lorentz transform (in slide) to calculate $\gamma$ 
and $\beta$. 

# [20]

show that the momentum 4-vector has an invariant length of $-(mc^2)^2$

Use $p^\mu = mu^\mu = (E/c, p_x c, p_y c, p_z c)$ and the expression $E = \sqrt{(mc^2)^2 + (pc)^2}$, taking normed length of $p^\mu$, to show invariant length of momentum 4-vector. 

$$ 
\|p^\mu\|_2 = \left\|
\begin{pmatrix} 
    \frac{1}{c} \sqrt{(mc^2)^2 + (pc)^2} \\
    p_x c \\
    p_y c \\
    p_z c 
\end{pmatrix} \right\|_2 \\

\Rightarrow p^\mu \eta_{\mu\nu} p^\nu = \left\|
\begin{pmatrix}
    -c^2 \sqrt{ m^2 c^2 + p^2} \\
    p_x c \\
    p_y c \\
    p_z c 
\end{pmatrix}\right\|_2 \\

\Rightarrow (p^\mu) ^ 2 = -c^2 ( m^2 c^2 + p^2 ) + (p_x c)^2 + (p_y c)^2 + (p_z c)^2 \\

\Rightarrow \|p^\mu\|_2 = -c^2 (m^2 c^2 + p^2) + (pc)^2 \\
\Rightarrow = -m^2 c^4 + (pc)^2 - (pc)^2
$$ 

$$ \rightarrow \|p^\mu\|_2 = p^\mu \eta_{\mu\nu} p^\nu = -(mc^2)^2

# [22]

Get the Ricci Tensor/scalar to show the field equation for $h_{\mu\nu}$ is correct

Recall that the approximation for the linearized gravitational field equations is 

$$ g_{\mu\nu} \approx \eta_{\mu\nu} + \kappa h_{\mu\nu} $$

and we are attempting to recreate the expression for linearized gravity in terms of 
$h_{\mu\nu}$ from the text, eqn. 3.38 from pp. 104 of the text:

$$\begin{aligned} 
\partial_\lambda \partial^\lambda h^{\mu\nu} + \partial^\mu \partial^\nu h \\
- \left( \partial_\lambda \partial^\nu h^{\mu\lambda} + \partial_\lambda \partial^\mu  h^{\nu\lambda}\right) \\
- \eta^{\mu\nu} \partial_\lambda \partial^\lambda h \\
+ \eta^{\mu\nu} \partial_\lambda \partial_\sigma h^{\lambda \sigma}

\end{aligned} = -\kappa T^{\mu\nu} \tag{3.38}$$

Note :> if we perform some simplification of the above statement, it is equivalent
to

$$ \begin{aligned} 
\partial^\mu \partial^\nu h \\
- \partial_\lambda \left( \partial^\nu h^{\mu\lambda} + \partial^\mu  h^{\nu\lambda}- \partial^\lambda h^{\mu\nu} \right) \\
+ \eta^{\mu\nu} \partial_\lambda \left[
\partial_\sigma h^{\lambda \sigma} -  \partial^\lambda h \right]

\end{aligned} = -\kappa T^{\mu\nu} 
$$


---

Since the Riemann (and therefore Ricci) Tensor is dependent on the metric, we
should start with the Christfoffel symbols for the choice of metric $h_{\mu\nu}$. 

$$
\Rightarrow \Gamma^\alpha_{\mu\nu} = 
\frac{1}{2}\ \mathrm{inv}\left( \eta_{\alpha \beta} + \kappa h_{\alpha\beta} \right) 
\left[ 
    (\eta_{\beta\mu,\nu} + \kappa h_{\beta\mu, \nu}) 
    + (\eta_{\nu\beta,\mu} + \kappa h_{\nu\beta, \mu})
    - (\eta_{\mu\nu,\beta} + \kappa h_{\mu\nu, \beta})
\right]
$$

We can then assume that $\eta_{\alpha \beta} \gg \kappa h_{\alpha \beta}$, so we will 
simplify the above to 

$$
\Rightarrow \Gamma^\alpha_{\mu\nu} = 
\frac{1}{2}\ \left( \eta^{\alpha \beta} \right) 
\left[ 
    (\eta_{\beta\mu,\nu} + \kappa h_{\beta\mu, \nu}) 
    + (\eta_{\nu\beta,\mu} + \kappa h_{\nu\beta, \mu})
    - (\eta_{\mu\nu,\beta} + \kappa h_{\mu\nu, \beta})
\right]
$$


Next, since the elements of the Minkowski metric are all constant, the partial
derivative terms above $\eta_{\mu\nu,\sigma} = 0$!  Therefore; 

$$
\Rightarrow \Gamma^\alpha_{\mu\nu} = 
\frac{1}{2}\ \kappa \left( \eta^{\alpha \beta} \right) 
\left[ 
    \partial_\nu h_{\beta\mu}
    + \partial_\mu h_{\nu\beta}
    - \partial_\beta h_{\mu\nu}
\right]
$$

A further simplification that we can make, since we are in the first-order (linearized) approximation, is to ignore terms in the defn. of the Riemann Tensor that result in higher-order terms of $h_{\mu\nu}$, so we will only consider

$$
\mathcal{R}^\alpha_{\beta\mu\nu} = - \partial_\nu \Gamma^\alpha_{\beta\mu} + \partial_\mu \Gamma^\alpha_{\beta\nu} + \ldots
$$

$$\Rightarrow 
\left\{ \begin{aligned}
\Gamma^\alpha_{\beta\mu} &= 
\frac{1}{2} \kappa \left( \eta^{\alpha\sigma} \right)
\left[ 
    \partial_\mu h_{\sigma \beta} 
    + \partial_\beta h_{\mu\sigma} 
    - \partial_\sigma h_{\beta\mu}
\right] \\
\Gamma^\alpha_{\beta\nu} &= 
\frac{1}{2} \kappa \left( \eta^{\alpha\sigma} \right)
\left[ 
    \partial_\nu h_{\sigma \beta} 
    + \partial_\beta h_{\nu\sigma} 
    - \partial_\sigma h_{\beta\nu}
\right] \\
\end{aligned} \right\}
$$

$$
\Rightarrow \mathcal{R}^\alpha_{\beta\mu\nu} = \frac{1}{2} \kappa \eta^{\alpha\sigma} 
\left( 
    - \partial_{\nu} 
    \left[ 
        \partial_\mu h_{\sigma \beta} 
        + \partial_\beta h_{\mu\sigma} 
        - \partial_\sigma h_{\beta\mu}
    \right]    
    + \partial_\mu 
    \left[ 
        \partial_\nu h_{\sigma \beta} 
        + \partial_\beta h_{\nu\sigma} 
        - \partial_\sigma h_{\beta\nu}
    \right]
\right)
$$

If we now distribute, simplify, and group by like terms; 

$$
\Rightarrow \mathcal{R}^\alpha_{\beta\mu\nu} = \frac{1}{2} \kappa \eta^{\alpha\sigma} 
\left( 
    \left[ 
        \partial_\mu \partial_\nu h_{\sigma \beta} 
        - \partial_{\nu} \partial_\mu h_{\sigma \beta} 
    \right]
    +
    \left[
        \partial_{\nu} \partial_\sigma h_{\beta\mu}
        - \partial_\mu \partial_\sigma h_{\beta\nu}
    \right]    
    +
    \left[ 
        \partial_\mu \partial_\beta h_{\nu\sigma} 
        - \partial_{\nu} \partial_\beta h_{\mu\sigma} 
    \right]
\right)
$$

We can now invoke that in this space, the derivative operator is commutative (i.e., $\partial_\mu \partial_\nu A - \partial_\nu \partial_\mu A = 0$), and we will find that some terms will cancel. 

$$
\Rightarrow \mathcal{R}^\alpha_{\beta\mu\nu} = \frac{1}{2} \kappa \eta^{\alpha\sigma} 
\left( 
    \left[
        \partial_{\nu} \partial_\sigma h_{\beta\mu}
        - \partial_\mu \partial_\sigma h_{\beta\nu}
    \right]    
    +
    \left[ 
        \partial_\mu \partial_\beta h_{\nu\sigma} 
        - \partial_{\nu} \partial_\beta h_{\mu\sigma} 
    \right]
\right)
$$

$$
\Rightarrow \mathcal{R}^\alpha_{\beta\mu\nu} = \frac{1}{2} \kappa \eta^{\alpha\sigma} 
\left(
    \partial_\sigma
    \left[
        \partial_{\nu}  h_{\beta\mu}
        - \partial_\mu h_{\beta\nu}
    \right]    
    +
    \partial_\beta
    \left[ 
        \partial_\mu  h_{\nu\sigma} 
        - \partial_{\nu} h_{\mu\sigma} 
    \right]    
\right)
$$

Now, if we perform the Ricci contraction ($\beta = \alpha$) on the above expression, 
we arrive at 

$$
\begin{aligned}
    \mathcal{R}_{\mu\nu} &= \mathcal{R}^\alpha_{\mu\nu\alpha} \\
    \Rightarrow 
    \mathcal{R}_{\mu\nu} &= 
\end{aligned}
$$

---

Once we have the expression for the Ricci Tensor and Curvature Scalar, plug directly into 
EHFE, and things should fall out. 

# [23]

Get $h_{\mu\nu}$ for sun and show that it satisfies the linearized equation, get geodesic equation for light from $h_{\mu\nu}$

# [24] 

Reproduce figure 4.5 from pg 140 of the text on the deflection of light.

# [25]

verify the solution of the scalar wave equation, the "plane wave" solution.

# [26]

Do Exercises 3, 4, and 5 in Section 3.1 of Ohanian.


## §3.1 ex. 3

>   Show that the gauge transformation (3.19) produces no change in $F_{\mu\nu}$.
>
>   $$ A^\nu \rightarrow A^\nu + \partial^\nu \Lambda \tag{3.19} $$
>
>   given: 
>
>   *   $$ F^{\mu\nu} \equiv \partial^\mu A^\nu - \partial^\nu A^\mu \tag{3.15} $$
>   *   $\Lambda$ is any arbitrary scalar function, the _gauge function_.  

## §3.1 ex. 4

>   Show that for a wave propagating in the $z$-direction, Eqs. (3.24) and (3.20), 
>   with $j^\nu = 0$, have the following linearly independent solutions:
>
>   $$ A^\nu = \epsilon_{(n)}^\nu \cos\left(\omega z - \omega t\right) \tag{3.26}$$
>
>   where $\epsilon_{(n)}^{\nu} : n \in \{1, 2, 3\}$ is one of the following vectors:
>
>   * $\epsilon_{(1)}^\nu = (0, 1, 0, 0)$
>   * $\epsilon_{(2)}^\nu = (0, 0, 1, 0)$
>   * $\epsilon_{(3)}^\nu = (1, 0, 0, 1)$
>
>   (Note :> The subscript $n$ has been enclosed in parentheses to indicate that it is 
>    not a tensor index; it is only a label that distinguishes between the four&ndash;
>    vectors above.)
>
>   Show that any other solution of the type $\cos\left(\omega z - \omega t\right)$
>   can be written as a linear combination of the three solutions, with the 
>   _polarization vectors_ $\epsilon_{(n)}^\nu$ listed above. 

## §3.1 ex. 5

>   Find the $\mathbf{E}$ and $\mathbf{B}$ fields for each of the solutions 
>   $\epsilon_{(n)}^\nu$, and show that the $\epsilon_{(3)}^\nu$ solution 
>   gives $\mathbf{E} = \mathbf{B} = 0$.
>
>   Hence, only the solutions with the first two polarization vectors $\epsilon_{(1)}^\nu$
>   and $\epsilon_{(2)}^\nu$ have any physical significance. 
>
>   These two polarization vectors correspond to the familiar two independent, 
>   perpendicular directions of polarization of an electromagnetic wave.  Show that
>   the solution with the polarization vector $\epsilon_{(3)}$\nu$ can be
>   completely eliminated by a gauge transformation, with $\Lambda = (1/\omega)
>   \sin\left(\omega z - \omega t\right)$, which leaves to $\overline{A}^\nu = 0$. 

# [27]

Compute the effect of a grav. wave traveling in the z-dir on a circle of masses arranged in the x-y plane.